<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/pf%2Fpre_covid_model/PF_sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("Sentiment").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment.csv"), sep=",", header=None)

# Show DataFrame
df.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
|Positive|Blue Shield of Ca...|
|Positive|RSX: Russian Equi...|
|Positive|XLB: Hold Your Po...|
|Positive|Musgrave Minerals...|
|Positive|ViacomCBS to rest...|
|Positive|Oil climbs to one...|
|Negative|Australian regula...|
|Positive|The Wall Street J...|
|Positive|Ora Banda Mining ...|
|Positive|Google Pay Plans ...|
|Negative|Toyota, Honda Ang...|
| Neutral|ZY Investor Alert...|
| Neutral|Calima Energy spu...|
|Positive|These Marijuana S...|
|Positive|Moderna ETFs to R...|
| Neutral|MacroGenics Annou...|
| Neutral|Zymeworks Announc...|
| Neutral|BioXcel Therapeut...|
| Neutral|Merck to Present ...|
| Neutral|BeiGene to Presen...|
+--------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Presen...|    88|
+--------+--------------------+------+
only showing top 20 rows



In [ ]:
# Rename columns
df1 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df1.printSchema()
df1.show()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Pre

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
pos_neg_to_num

StringIndexer_f56c7fa6f224

In [ ]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="tokenized")
tokenized = tokenizer.transform(df1)

In [ ]:
# Remove stop words
stop_list = ["Sepp+�l+�", "EUR", ".", "%", "eur", "mn", "m", "?", "$", "#", "@"]
stopremove = StopWordsRemover(inputCol='tokenized',outputCol='stopremove', stopWords=stop_list)
removed_df = stopremove.transform(tokenized)
removed_df.show(truncate=False)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class   |text                                                                                                                                                                         |length|tokenized                                                                                                                                                                                          |stopremove                   

In [ ]:
# Run the hashing term frequency
hashingTF = HashingTF(inputCol="stopremove", outputCol='hash_token')

In [ ]:
# Fit the IDF on the data set
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df1)
cleaned = cleaner.transform(df1)

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[43204,50...|
|  1.0|(262145,[31536,58...|
|  1.0|(262145,[619,5765...|
|  1.0|(262145,[57093,58...|
|  1.0|(262145,[27576,38...|
|  1.0|(262145,[15862,27...|
|  2.0|(262145,[6993,206...|
|  1.0|(262145,[38308,63...|
|  1.0|(262145,[22459,27...|
|  1.0|(262145,[66208,84...|
|  2.0|(262145,[3091,942...|
|  0.0|(262145,[102692,1...|
|  0.0|(262145,[9129,233...|
|  1.0|(262145,[16004,22...|
|  1.0|(262145,[27576,49...|
|  0.0|(262145,[18731,22...|
|  0.0|(262145,[31793,45...|
|  0.0|(262145,[1097,216...|
|  0.0|(262145,[2690,275...|
|  0.0|(262145,[26190,27...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|           tokenized|          stopremove|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Negative|"Apple rejects Ep...|    72|  2.0|["apple, rejects,...|["apple, rejects,...|(262144,[15625,38...|(262144,[15625,38...|(262145,[15625,38...|[-879.68136638752...|[1.0,1.6859614660...|       0.0|
|Negative|2 Warren Buffett ...|    51|  2.0|[2, warren, buffe...|[2, warren, buffe...|(262144,[12524,27...|(262144,[12524,27...|(262145,[12524,27...|[-656.67126235904...|[5.15520794365627.

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.686726


In [ ]:
url1 = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/pre-covid-test-data.csv'
spark.sparkContext.addFile(url1)
pre_covid_df = spark.read.csv(SparkFiles.get("pre-covid-test-data.csv"), sep=",", header=None)

# Show DataFrame
pre_covid_df.show()

+--------------------+
|                 _c0|
+--------------------+
|Stitch Fix, Inc. ...|
|Goldman Sachs res...|
|What We Learned F...|
|Cerner: A Great S...|
|Johnson & Johnson...|
|5 Key Conclusions...|
|Foot Locker: Wort...|
|The Stock Market ...|
|These High-Yield ...|
|This High-Yield S...|
|New Oriental Educ...|
|J&J to pay $20.4 ...|
|Pharma Giant J&J ...|
|Brokerage Stocks ...|
|Johnson & Johnson...|
|Visa, Mastercard ...|
|Apple Looking For...|
|Why Cloud Computi...|
|Micron: Back To R...|
|The New Ireland F...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df1 = pre_covid_df.withColumn('length', length(pre_covid_df['_c0']))
data_df1.show()

+--------------------+------+
|                 _c0|length|
+--------------------+------+
|Stitch Fix, Inc. ...|    86|
|Goldman Sachs res...|    71|
|What We Learned F...|    56|
|Cerner: A Great S...|    54|
|Johnson & Johnson...|    62|
|5 Key Conclusions...|    63|
|Foot Locker: Wort...|    29|
|The Stock Market ...|    63|
|These High-Yield ...|    50|
|This High-Yield S...|    74|
|New Oriental Educ...|    76|
|J&J to pay $20.4 ...|    73|
|Pharma Giant J&J ...|    72|
|Brokerage Stocks ...|    56|
|Johnson & Johnson...|    66|
|Visa, Mastercard ...|    65|
|Apple Looking For...|    26|
|Why Cloud Computi...|    60|
|Micron: Back To R...|    23|
|The New Ireland F...|    52|
+--------------------+------+
only showing top 20 rows



In [ ]:
# Rename columns
df2 = data_df1.withColumnRenamed("_c0", "text")
df2.printSchema()
df2.show()

root
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------------------+------+
|                text|length|
+--------------------+------+
|Stitch Fix, Inc. ...|    86|
|Goldman Sachs res...|    71|
|What We Learned F...|    56|
|Cerner: A Great S...|    54|
|Johnson & Johnson...|    62|
|5 Key Conclusions...|    63|
|Foot Locker: Wort...|    29|
|The Stock Market ...|    63|
|These High-Yield ...|    50|
|This High-Yield S...|    74|
|New Oriental Educ...|    76|
|J&J to pay $20.4 ...|    73|
|Pharma Giant J&J ...|    72|
|Brokerage Stocks ...|    56|
|Johnson & Johnson...|    66|
|Visa, Mastercard ...|    65|
|Apple Looking For...|    26|
|Why Cloud Computi...|    60|
|Micron: Back To R...|    23|
|The New Ireland F...|    52|
+--------------------+------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
pos_neg_to_num

StringIndexer_2dab5ed86bad

In [ ]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="tokenized")
tokenized = tokenizer.transform(df2)

In [ ]:
# Remove stop words
stop_list1 = ["Sepp+�l+�", "EUR", ".", "%", "eur", "mn", "m", "?", "$", "#", "@"]
stopremove1 = StopWordsRemover(inputCol='tokenized',outputCol='stopremove1', stopWords=stop_list1)
removed_df1 = stopremove1.transform(tokenized)
removed_df1.show(truncate=False)

+--------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------+
|text                                                                                  |length|tokenized                                                                                             |stopremove1                                                                                           |
+--------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------+
|Stitch Fix, Inc. (SFIX) CEO Katrina Lake on Q4 2019 Results - Earnings Call Transcript|86    

In [ ]:
# Run the hashing term frequency
hashingTF1 = HashingTF(inputCol="stopremove1", outputCol='hash_token1')

In [ ]:
# Fit the IDF on the data set
idf1 = IDF(inputCol='hash_token1', outputCol='idf_token1')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up1 = VectorAssembler(inputCols=['idf_token1', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline1 = Pipeline(stages=[tokenizer, stopremove1, hashingTF1, idf1, clean_up1])

In [ ]:
# Fit and transform the pipeline
cleaner1 = data_prep_pipeline1.fit(df2)
cleaned1 = cleaner.transform(df2)

In [ ]:
# Show label and resulting features
cleaned1.select(['features']).show()

+--------------------+
|            features|
+--------------------+
|(262145,[6012,339...|
|(262145,[17470,17...|
|(262145,[32108,81...|
|(262145,[18697,27...|
|(262145,[2162,397...|
|(262145,[101169,1...|
|(262145,[44541,51...|
|(262145,[8145,125...|
|(262145,[22575,27...|
|(262145,[18697,54...|
|(262145,[70835,83...|
|(262145,[3973,266...|
|(262145,[2162,340...|
|(262145,[31462,89...|
|(262145,[2162,504...|
|(262145,[8981,283...|
|(262145,[89833,10...|
|(262145,[5835,403...|
|(262145,[27576,51...|
|(262145,[27506,89...|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import lit
cleaned2 = cleaned1.withColumn("label", lit(0))
cleaned2.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|                text|length|           tokenized|          stopremove|          hash_token|           idf_token|            features|label|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|Stitch Fix, Inc. ...|    86|[stitch, fix,, in...|[stitch, fix,, in...|(262144,[6012,339...|(262144,[6012,339...|(262145,[6012,339...|    0|
|Goldman Sachs res...|    71|[goldman, sachs, ...|[goldman, sachs, ...|(262144,[17470,17...|(262144,[17470,17...|(262145,[17470,17...|    0|
|What We Learned F...|    56|[what, we, learne...|[what, we, learne...|(262144,[32108,81...|(262144,[32108,81...|(262145,[32108,81...|    0|
|Cerner: A Great S...|    54|[cerner:, a, grea...|[cerner:, a, grea...|(262144,[18697,27...|(262144,[18697,27...|(262145,[18697,27...|    0|
|Johnson & Jo

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned2.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
# predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results1 = predictor.transform(testing)
test_results1.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|                text|length|           tokenized|          stopremove|          hash_token|           idf_token|            features|label|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|"Comcast Says Cor...|    86|["comcast, says, ...|["comcast, says, ...|(262144,[12274,27...|(262144,[12274,27...|(262145,[12274,27...|    0|[-947.49803609039...|[0.90622427338317...|       0.0|
|   "How ""Buy Online|    17|["how, ""buy, onl...|["how, ""buy, onl...|(262144,[50671,79...|(262144,[50671,79...|(262145,[50671,79...|    0|[-299.91137245314...|[0.98312346313230...|       0.0|
|"Smart Timing for...|    98|[

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval1 = MulticlassClassificationEvaluator()
acc = acc_eval1.evaluate(test_results1)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.807133
